In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import os


## Collecting data from Forbes using BeautifulSoup:

Let's look at the structure in which our data is stored. We will use the BeautifulSoup library to parse the HTML code of the page. We will use the `requests` library to get the HTML code of the page.

In [ ]:
i = 1
folder = '14datafiles'
file_name = f'page{i}.txt'
path = os.path.join(folder, file_name)

with open(path, 'r') as file:
    content = file.read()
    soup = BeautifulSoup(content, 'html.parser')

    bunch = soup.find('div', {'class': 'TableRow_row__L-0Km'})
    print(bunch.prettify(), '\n\n')

    elements = bunch.find_all('div', {'class': 'TableRow_cell__db-hv'})
    print(elements, '\n\n')

    for _ in elements:
        print(_.text.strip())

    rank = int(elements[0].text.strip())
    forbes_id = bunch['id'].strip()
    name = elements[1].text.strip()
    net_worth = int(elements[2].text.strip(' B$'))
    age = int(elements[3].text.strip())
    country = elements[4].text.strip()
    source = elements[5].text.strip()
    industry = elements[6].text.strip()

    print('\n\n', type(rank),'|||', forbes_id,'|||', name,'|||', type(net_worth),'|||', type(age),'|||', country,'|||', source,'|||', industry)

<div class="TableRow_row__L-0Km" id="bernard-arnault" role="row" style="display: flex; flex: 1 0 auto; min-width: 0px;">
 <div class="TableRow_cell__db-hv Table_cell__houv9" role="cell" style="box-sizing: border-box; flex: 150 0 auto; min-width: 0px; width: 150px;">
  <div class="Table_rank___YBhk Table_dataCell__2QCve">
   1
  </div>
 </div>
 <div class="TableRow_cell__db-hv Table_cell__houv9" role="cell" style="box-sizing: border-box; flex: 150 0 auto; min-width: 0px; width: 150px;">
  <div class="Table_dataCell__2QCve">
   Bernard Arnault &amp; family
  </div>
 </div>
 <div class="TableRow_cell__db-hv Table_cell__houv9" role="cell" style="box-sizing: border-box; flex: 150 0 auto; min-width: 0px; width: 150px;">
  <div class="Table_netWorth___L4R5 Table_dataCell__2QCve">
   $211 B
   <div class="Table_status__zATxt">
    <div class="WealthIcon_wealthStatus__n-T5L">
     <img alt="up arrow" class="WealthIcon_wealthIcon__aW6BL" src="https://images.forbes.com/assets/list-landers/forbes-

I have copied all the HTML code of 14 pages and saved it in a folder called `14datafiles`. It contains 14 text files and each file has the block of HTML which contain list if billionares. We will use these files to parse the HTML code.

Fisrt we will check the HTML code of one file and then we will write a function to parse all the files.

In [ ]:
dict = {'rank': [],
        'name': [],
        'net_worth': [],
        'age': [],
        'age_range': [],
        'country': [],
        'source': [],
        'industry': []
        }
df_test = pd.DataFrame(dict)

i = 1
folder = '14datafiles'
file_name = f'page{i}.txt'
path = os.path.join(folder, file_name)

with open(path, 'r') as file:

    content = file.read()
    soup = BeautifulSoup(content, 'html.parser')

    bunch = soup.find_all('div', {'class': 'TableRow_row__L-0Km'})

    for bunch_single in bunch:
        elements = bunch_single.find_all('div', {'class': 'TableRow_cell__db-hv'})

        rank = int(elements[0].text.strip())
        forbes_id = bunch_single['id'].strip()
        name = elements[1].text.strip()
        net_worth = float(elements[2].text.strip(' B$'))

        age = elements[3].text.strip()
        if age == 'N/A':
            age = np.NaN
        else:
            age = re.findall(r'\d+', age)
            age = [int(_) for _ in age]
            age = age[0]
        
        if age == np.NaN:
            age_range = 0
        else:
            for i in range(1, 11):
                if age > (i-1)*10 and age <= i*10:
                    age_range = i
                    break
            if age > 100:   
                age_range = 11
                
        country = elements[4].text.strip()
        source = elements[5].text.strip()
        industry = elements[6].text.strip()

        df_test.loc[len(df_test.index)] = [rank, name, net_worth, age, age_range, country, source, industry]
df_test

,rank,name,net_worth,age,age_range,country,source,industry
0,1,Bernard Arnault & family,211.0,74.0,8,France,LVMH,Fashion & Retail
1,2,Elon Musk,180.0,51.0,6,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,114.0,59.0,6,United States,Amazon,Technology
3,4,Larry Ellison,107.0,78.0,8,United States,Oracle,Technology
4,5,Warren Buffett,106.0,92.0,10,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...,...
195,195,Jin Baofang,9.6,70.0,7,China,Solar panels,Energy
196,195,Luo Liguo & family,9.6,67.0,7,China,Chemicals,Manufacturing
197,195,Marijke Mars,9.6,58.0,6,United States,"Candy, pet food",Food & Beverage
198,195,Pamela Mars,9.6,62.0,7,United States,"Candy, pet food",Food & Beverage


We got the data of first file successfully. Now we will write a function to parse all the files.

As you can in below code I have commented the code which catagorizes people in `age_ranges`. That is because it was increasing the time of execution. so then I have write that code saparately.

In [ ]:
dict = {'rank': [],
        'name': [],
        'forbes_id': [],
        'net_worth': [],
        'age': [],
        'age_range': [],
        'country': [],
        'source': [],
        'industry': []
        }
df = pd.DataFrame(dict)

i = 1; k = 14
folder = '14datafiles'

while (i<=k):

    file_name = f'page{i}.txt'
    path = os.path.join(folder, file_name)

    with open(path, 'r') as file:
        content = file.read()
        soup = BeautifulSoup(content, 'html.parser')

        bunch = soup.find_all('div', {'class': 'TableRow_row__L-0Km'})

        for bunch_single in bunch:
            elements = bunch_single.find_all('div', {'class': 'TableRow_cell__db-hv'})

            rank = int(elements[0].text.strip())
            forbes_id = bunch_single['id'].strip() if bunch_single['id'] else None
            name = elements[1].text.strip()
            net_worth = float(elements[2].text.strip(' B$'))
            age = elements[3].text.strip()
            if age == 'N/A':
                age = np.NaN
            else:
                age = re.findall(r'\d+', age)
                age = [int(_) for _ in age]
                age = age[0]
            
            age_range = 0
            # if age == np.NaN:
            #     age_range = 0
            # else:
            #     for i in range(1, 11):
            #         if age > (i-1)*10 and age <= i*10:
            #             age_range = i
            #             break
            #     if age > 100:   
            #         age_range = 11

            country = elements[4].text.strip()
            source = elements[5].text.strip()
            industry = elements[6].text.strip()

            df.loc[len(df.index)] = [rank, name, forbes_id, net_worth, age, age_range, country, source, industry]
    i+=1
df

,rank,name,forbes_id,net_worth,age,age_range,country,source,industry
0,1,Bernard Arnault & family,bernard-arnault,211.0,74.0,0,France,LVMH,Fashion & Retail
1,2,Elon Musk,elon-musk,180.0,51.0,0,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,jeff-bezos,114.0,59.0,0,United States,Amazon,Technology
3,4,Larry Ellison,larry-ellison,107.0,78.0,0,United States,Oracle,Technology
4,5,Warren Buffett,warren-buffett,106.0,92.0,0,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...,...,...
2635,2540,Yu Rong,yu-rong,1.0,51.0,0,China,Health clinics,Healthcare
2636,2540,"Richard Yuengling, Jr.",richard-yuengling-jr,1.0,80.0,0,United States,Beer,Food & Beverage
2637,2540,Zhang Gongyun,zhang-gongyun,1.0,60.0,0,China,Tyre manufacturing machinery,Manufacturing
2638,2540,Zhang Guiping & family,zhang-guiping,1.0,71.0,0,China,Real estate,Real Estate


In [ ]:
for i in df.index:
    if df.loc[i, 'age'] == np.NaN:
        df.loc[i, 'age_range'] = 0
    else:
        for j in range(1, 11):
            if df.loc[i, 'age'] > (j-1)*10 and df.loc[i, 'age'] <= j*10:
                df.loc[i, 'age_range'] = j
                break
        if df.loc[i, 'age'] > 100:
            df.loc[i, 'age_range'] = 11
df

,rank,name,forbes_id,net_worth,age,age_range,country,source,industry
0,1,Bernard Arnault & family,bernard-arnault,211.0,74.0,8,France,LVMH,Fashion & Retail
1,2,Elon Musk,elon-musk,180.0,51.0,6,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,jeff-bezos,114.0,59.0,6,United States,Amazon,Technology
3,4,Larry Ellison,larry-ellison,107.0,78.0,8,United States,Oracle,Technology
4,5,Warren Buffett,warren-buffett,106.0,92.0,10,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...,...,...
2635,2540,Yu Rong,yu-rong,1.0,51.0,6,China,Health clinics,Healthcare
2636,2540,"Richard Yuengling, Jr.",richard-yuengling-jr,1.0,80.0,8,United States,Beer,Food & Beverage
2637,2540,Zhang Gongyun,zhang-gongyun,1.0,60.0,6,China,Tyre manufacturing machinery,Manufacturing
2638,2540,Zhang Guiping & family,zhang-guiping,1.0,71.0,8,China,Real estate,Real Estate


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2640 entries, 0 to 2639
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       2640 non-null   int64  
 1   name       2640 non-null   object 
 2   forbes_id  2640 non-null   object 
 3   net_worth  2640 non-null   float64
 4   age        2575 non-null   float64
 5   age_range  2640 non-null   int64  
 6   country    2640 non-null   object 
 7   source     2640 non-null   object 
 8   industry   2640 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 270.8+ KB


Now we are going to collect the `more` data of billionaires from each ones `individual profile` on the Forbes website.

Below is the code to see the structure of the HTML which contains the required data of the billionaire.

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of the website you want to scrape
billionaire = ['elon-musk', 'warren-buffett']

for billionaire in billionaire:
    url = f'https://www.forbes.com/profile/{billionaire}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    detail = soup.find('dl', {'class': 'listuser-block'})
    print("First paragraph:", detail.prettify())


First paragraph: <dl class="listuser-block">
 <dl class="listuser-block__item">
  <dt class="profile-stats__title">
   Age
  </dt>
  <dd class="profile-stats__text">
   <span class="profile-stats__text">
    52
   </span>
  </dd>
 </dl>
 <dl class="listuser-block__item">
  <dt class="profile-stats__title">
   Source of Wealth
  </dt>
  <dd class="profile-stats__text">
   <span class="profile-stats__text">
    Tesla, SpaceX, Self Made
   </span>
  </dd>
 </dl>
 <dl class="listuser-block__item">
  <dt class="profile-stats__title">
   <a data-profile-ga-track="self made score-personal stats-Elon Musk" href="https://www.forbes.com/self-made-methodology/">
    Self-Made Score
   </a>
  </dt>
  <dd class="profile-stats__text">
   <span class="profile-stats__text">
    8
   </span>
  </dd>
 </dl>
 <dl class="listuser-block__item">
  <dt class="profile-stats__title">
   <a data-profile-ga-track="philanthropy score-personal stats-Elon Musk" href="https://www.forbes.com/philanthropy-methodology/

In [ ]:
# Dict = {'Age': [], 'Source of Wealth': [], 'Self-Made Score': [], 'Philanthropy Score': [], 'Residence': [], 'Citizenship': [], 'Marital Status': [], 'Children': [], 'Education': []}
# billionaire_index = 0

In [ ]:
# import requests
# from bs4 import BeautifulSoup

# for billionaire in df['forbes_id'][0:10]:
#     try:
#         url = f'https://www.forbes.com/profile/{billionaire}'
#         response = requests.get(url)
#         soup = BeautifulSoup(response.content, 'html.parser')

#         bunch = soup.find('dl', {'class': 'listuser-block'})
#         # print(bunch.prettify(), '\n\n')

#         elements = bunch.find_all('dl', {'class':"listuser-block__item"})
#         # print(elements, '\n\n')

#         for _ in Dict:
#             Dict[_].append(np.NaN)

#         for _ in elements:
#             # print(_.text.strip())
#             Dict[_.find('dt').text.strip()][billionaire_index] = (_.find('span').text.strip())

#         billionaire_index += 1
#     except:
#         for _ in Dict:
#             Dict[_].append('error')
#         billionaire_index += 1
#         continue
# print(Dict)

{'Age': ['74', '52', '59', '79', '92', '67', '81', '83', '66', '67', '70', '50', '87', '50', '68', '39', '87', '61', '75', '78', '73', '66', '58', '61', '85', '39', '83', '87', '86', '58', '83', '87', '95', '51', '77', '54', '31', '54', '74', '66', '74', '72', '83', '78', '43', '88', '95', '65', '85', '76', '61', '69', '51', '69', '78', '75', '78', '51', '57', '53', '82', '62', '58', '81', '80', '83', '70', '82', '66', '67', '52', '61', '68', '36', '52', '60', '90', '78', '57', '72', '85', '61', '74', '60', '55', '73', '58', '57', '77', '65', '70', '61', '73', '67', '87', '73', '91', '83', '92', '70', '72', '56', '72', '75', '68', '55', '67', nan, nan, '84', '76', '67', '45', '68', '56', '84', '81', '79', '69', '61', '59', '69', '96', '56', '66', '93', '67', '57', '94', '78', '76', '47', '80', '81', '72', '60', '90', '64', '76', '59', '58', '50', '69', '73', '59', '99', '83', '38', '73', '73', '59', '51', '74', '69', '63', '58', '89', '73', '58', '51', '83', '86', '67', '94', '82', '73

In [ ]:
# df_extra = pd.DataFrame(Dict)
# df_extra
# df_extra.tail(15)

# # we are going to remove last 7 rows because they are not billionaires

# df_extra = df_extra[:-12]
# df_extra


,Age,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education
0,74,LVMH,NaN,NaN,"Paris, France",France,Married,5,"Bachelor of Arts/Science, Ecole Polytechnique ..."
1,52,"Tesla, SpaceX, Self Made",8,1,"Austin, Texas",United States,Single,10,"Bachelor of Arts/Science, University of Pennsy..."
2,59,"Amazon, Self Made",8,2,"Medina, Washington",United States,Engaged,4,"Bachelor of Arts/Science, Princeton University"
3,79,"Oracle, Self Made",9,1,"Lanai, Hawaii",United States,In Relationship,4,"Drop Out, University of Chicago; Drop Out, Uni..."
4,92,"Berkshire Hathaway, Self Made",8,5,"Omaha, Nebraska",United States,"Widowed, Remarried",3,"Master of Science, Columbia University; Bachel..."
...,...,...,...,...,...,...,...,...,...
2635,51,"Health clinics, Self Made",NaN,NaN,"Shanghai, China",China,NaN,NaN,NaN
2636,80,Beer,5,NaN,"Pottsville, Pennsylvania",United States,Divorced,4,"Drop Out, Lycoming College"
2637,60,"Tyre manufacturing machinery, Self Made",NaN,NaN,"Gaomi, China",China,NaN,NaN,NaN
2638,72,"Real estate, Self Made",NaN,NaN,"Nanjing, China",China,Married,1,"Bachelor of Arts/Science, South East University"


## Clean and Preprocess Data:

In [ ]:
# df_extra.isna().all(axis=1).sum()

# mask = df_extra.isna().all(axis=1)

# df_extra[2486:2490]

,Age,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education
2486,70,Motorcycles,NaN,NaN,"Delhi, India",India,Married,NaN,NaN
2487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2489,54,"Retail, Self Made",NaN,NaN,"Bacau, Romania",Romania,Married,NaN,NaN


In [ ]:
df.iloc[2486:2490]

,rank,name,forbes_id,net_worth,age,age_range,country,source,industry
2486,2405,Suman Munjal,suman-munjal,1.1,70.0,7,India,Motorcycles,Automotive
2487,2405,Dmitry Nikolaev,dmitry-nikolaev,1.1,56.0,6,Russia,Coal,Energy
2488,2405,Sjamsul Nursalim,sjamsul-nursalim,1.1,81.0,9,Indonesia,"Tires, retail",Automotive
2489,2405,Adrian Paval,adrian-paval,1.1,54.0,6,Romania,Retail,Fashion & Retail


In [ ]:
df_extra.loc[2487] = ['57', 'Coal, Self Made', np.NaN, np.NaN, 'Kemerovo, Russia', 'Russia', 'Married', '3', np.NaN]
df_extra.loc[2488] = ['81', 'Tires, retail', np.NaN, np.NaN, 'Singapore, Singapore', 'Indonesia', 'Married', '3', np.NaN]

df_extra[2486:2490]

,Age,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education
2486,70,Motorcycles,NaN,NaN,"Delhi, India",India,Married,NaN,NaN
2487,57,"Coal, Self Made",NaN,NaN,"Kemerovo, Russia",Russia,Married,3,NaN
2488,81,"Tires, retail",NaN,NaN,"Singapore, Singapore",Indonesia,Married,3,NaN
2489,54,"Retail, Self Made",NaN,NaN,"Bacau, Romania",Romania,Married,NaN,NaN


In [ ]:
df_extra

,Age,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education
0,74,LVMH,NaN,NaN,"Paris, France",France,Married,5,"Bachelor of Arts/Science, Ecole Polytechnique ..."
1,52,"Tesla, SpaceX, Self Made",8,1,"Austin, Texas",United States,Single,10,"Bachelor of Arts/Science, University of Pennsy..."
2,59,"Amazon, Self Made",8,2,"Medina, Washington",United States,Engaged,4,"Bachelor of Arts/Science, Princeton University"
3,79,"Oracle, Self Made",9,1,"Lanai, Hawaii",United States,In Relationship,4,"Drop Out, University of Chicago; Drop Out, Uni..."
4,92,"Berkshire Hathaway, Self Made",8,5,"Omaha, Nebraska",United States,"Widowed, Remarried",3,"Master of Science, Columbia University; Bachel..."
...,...,...,...,...,...,...,...,...,...
2635,51,"Health clinics, Self Made",NaN,NaN,"Shanghai, China",China,NaN,NaN,NaN
2636,80,Beer,5,NaN,"Pottsville, Pennsylvania",United States,Divorced,4,"Drop Out, Lycoming College"
2637,60,"Tyre manufacturing machinery, Self Made",NaN,NaN,"Gaomi, China",China,NaN,NaN,NaN
2638,72,"Real estate, Self Made",NaN,NaN,"Nanjing, China",China,Married,1,"Bachelor of Arts/Science, South East University"


In [ ]:
df_extra.to_csv('df_extra.csv', index=False)

In [ ]:
Education = {'Bachelor': [], 'Master': [], 'Doctorate': [], 'Drop Out': []}

for i in df_extra.index:
    for _ in Education:
        if type(df_extra['Education'][i]) == float:
            Education[_].append(np.NaN)
        else:
            if _ in df_extra['Education'][i]:
                Education[_].append(1)
            else:
                Education[_].append(0)

print(Education) 

{'Bachelor': [1, 1, 1, 0, 1, 0, 1, 1, 1, 1, nan, 1, nan, 1, nan, 0, 1, 1, 1, 1, 1, 0, 0, nan, 1, 1, nan, 0, nan, nan, 1, 1, 0, 1, 1, 1, nan, 0, 1, 1, nan, nan, nan, nan, 1, nan, nan, 1, 1, 1, 0, nan, 1, nan, 1, nan, 0, 0, 0, 1, nan, 0, 1, nan, nan, nan, nan, 1, 1, 1, nan, 1, 0, 1, nan, 1, 1, nan, 0, 0, 1, 1, 1, 0, nan, nan, nan, 1, 0, 1, 0, 1, 1, 1, 1, nan, 1, 1, 1, nan, nan, nan, 1, 0, 1, 1, nan, nan, nan, 1, 0, 1, 1, nan, 1, nan, 0, nan, 1, 1, 1, 0, nan, 1, 1, nan, nan, 0, nan, nan, 0, 0, 1, 1, nan, nan, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, nan, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, nan, 0, 1, 1, 1, 1, 0, 1, nan, 0, 1, 0, 1, 0, nan, nan, nan, nan, 1, nan, 0, 1, 1, 1, nan, 1, nan, 0, nan, 1, 1, 1, 1, 0, nan, 1, 0, 1, nan, 0, 0, 0, 1, 1, 1, nan, 1, 1, 1, 1, nan, nan, nan, nan, 1, 1, 1, 0, 1, nan, nan, 1, 0, 1, nan, nan, 0, 1, nan, 0, 1, 0, nan, 1, 1, 1, nan, 1, 1, nan, nan, 1, nan, 1, 1, 0, 1, 0, nan, 0, 0, 0, 1, 0, 1, 1, 1, 0, nan, 1, 0, 1, 1, 1, 1, 0, nan, 1, nan, 0

In [ ]:
Self_Made = {'Self Made':[]}

for i in df_extra.index:
    if 'Self Made' in df_extra['Source of Wealth'][i]:
        Self_Made['Self Made'].append(1)
    else:
        Self_Made['Self Made'].append(0)
  
print(Self_Made)

{'Self Made': [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0

In [ ]:
# we are going to add Education and Self_Made to df_extra
df_extra['Bachelor'] = Education['Bachelor']
df_extra['Master'] = Education['Master']
df_extra['Doctorate'] = Education['Doctorate']
df_extra['Drop Out'] = Education['Drop Out']
df_extra['Self Made'] = Self_Made['Self Made']

df_extra

,Age,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education,Bachelor,Master,Doctorate,Drop Out,Self Made
0,74,LVMH,NaN,NaN,"Paris, France",France,Married,5,"Bachelor of Arts/Science, Ecole Polytechnique ...",1.0,0.0,0.0,0.0,0
1,52,"Tesla, SpaceX, Self Made",8,1,"Austin, Texas",United States,Single,10,"Bachelor of Arts/Science, University of Pennsy...",1.0,0.0,0.0,0.0,1
2,59,"Amazon, Self Made",8,2,"Medina, Washington",United States,Engaged,4,"Bachelor of Arts/Science, Princeton University",1.0,0.0,0.0,0.0,1
3,79,"Oracle, Self Made",9,1,"Lanai, Hawaii",United States,In Relationship,4,"Drop Out, University of Chicago; Drop Out, Uni...",0.0,0.0,0.0,1.0,1
4,92,"Berkshire Hathaway, Self Made",8,5,"Omaha, Nebraska",United States,"Widowed, Remarried",3,"Master of Science, Columbia University; Bachel...",1.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,51,"Health clinics, Self Made",NaN,NaN,"Shanghai, China",China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2636,80,Beer,5,NaN,"Pottsville, Pennsylvania",United States,Divorced,4,"Drop Out, Lycoming College",0.0,0.0,0.0,1.0,0
2637,60,"Tyre manufacturing machinery, Self Made",NaN,NaN,"Gaomi, China",China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2638,72,"Real estate, Self Made",NaN,NaN,"Nanjing, China",China,Married,1,"Bachelor of Arts/Science, South East University",1.0,0.0,0.0,0.0,1


In [ ]:
# we are going to make a new dataframe with df and df_extra of 2640 billionaires

df_new = pd.concat([df, df_extra], axis=1)
df_new

,rank,name,forbes_id,net_worth,age,age_range,country,source,industry,Age,...,Residence,Citizenship,Marital Status,Children,Education,Bachelor,Master,Doctorate,Drop Out,Self Made
0,1,Bernard Arnault & family,bernard-arnault,211.0,74.0,8,France,LVMH,Fashion & Retail,74,...,"Paris, France",France,Married,5,"Bachelor of Arts/Science, Ecole Polytechnique ...",1.0,0.0,0.0,0.0,0
1,2,Elon Musk,elon-musk,180.0,51.0,6,United States,"Tesla, SpaceX",Automotive,52,...,"Austin, Texas",United States,Single,10,"Bachelor of Arts/Science, University of Pennsy...",1.0,0.0,0.0,0.0,1
2,3,Jeff Bezos,jeff-bezos,114.0,59.0,6,United States,Amazon,Technology,59,...,"Medina, Washington",United States,Engaged,4,"Bachelor of Arts/Science, Princeton University",1.0,0.0,0.0,0.0,1
3,4,Larry Ellison,larry-ellison,107.0,78.0,8,United States,Oracle,Technology,79,...,"Lanai, Hawaii",United States,In Relationship,4,"Drop Out, University of Chicago; Drop Out, Uni...",0.0,0.0,0.0,1.0,1
4,5,Warren Buffett,warren-buffett,106.0,92.0,10,United States,Berkshire Hathaway,Finance & Investments,92,...,"Omaha, Nebraska",United States,"Widowed, Remarried",3,"Master of Science, Columbia University; Bachel...",1.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2540,Yu Rong,yu-rong,1.0,51.0,6,China,Health clinics,Healthcare,51,...,"Shanghai, China",China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2636,2540,"Richard Yuengling, Jr.",richard-yuengling-jr,1.0,80.0,8,United States,Beer,Food & Beverage,80,...,"Pottsville, Pennsylvania",United States,Divorced,4,"Drop Out, Lycoming College",0.0,0.0,0.0,1.0,0
2637,2540,Zhang Gongyun,zhang-gongyun,1.0,60.0,6,China,Tyre manufacturing machinery,Manufacturing,60,...,"Gaomi, China",China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2638,2540,Zhang Guiping & family,zhang-guiping,1.0,71.0,8,China,Real estate,Real Estate,72,...,"Nanjing, China",China,Married,1,"Bachelor of Arts/Science, South East University",1.0,0.0,0.0,0.0,1


In [ ]:
# df_new.to_csv('forbes_2640_billionaires.csv', index=False)

In [ ]:
df = df_new

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2640 entries, 0 to 2639
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   rank                2640 non-null   int64  
 1   name                2640 non-null   object 
 2   forbes_id           2640 non-null   object 
 3   net_worth           2640 non-null   float64
 4   age                 2575 non-null   float64
 5   age_range           2640 non-null   int64  
 6   country             2640 non-null   object 
 7   source              2640 non-null   object 
 8   industry            2640 non-null   object 
 9   Age                 2556 non-null   object 
 10  Source of Wealth    2640 non-null   object 
 11  Self-Made Score     563 non-null    object 
 12  Philanthropy Score  418 non-null    object 
 13  Residence           2581 non-null   object 
 14  Citizenship         2629 non-null   object 
 15  Marital Status      2090 non-null   object 
 16  Childr

Few days after saving and then reading the csv to further move on. I have noticed that there are some missing values in the data. So I have to clean the data first.

In [ ]:
df = pd.read_csv('forbes_2640_billionaires.csv')

In [ ]:
# we are going to change values to nan in rows with 'error' value

df_error = df[df['Children'] == 'error']
df_error
for i in df_error.index:
    df.loc[i, 'Children'] = np.NaN

In [ ]:
print(df['Children'].unique())

['5' '10' '4' '3' '2' '6' '1' nan '7' '9' '8' '14' '15' '22' '11' '23']


In [ ]:
# we are going to change datatypes of Age, Self-Made Score, Philanthropy Score, Children to float

df['Age'] = df['Age'].astype(float)
df['Self-Made Score'] = df['Self-Made Score'].astype(float)
df['Philanthropy Score'] = df['Philanthropy Score'].astype(float)
df['Children'] = df['Children'].astype(float)

In [ ]:
#drop age column

df.drop('age', axis=1, inplace=True)